In [149]:
import itertools

import matplotlib.pyplot as plt
import numpy as np


def average_sequence(balls) -> int:
    rng = np.random.default_rng()
    s = 0
    for i in itertools.count(1):
        s += balls[rng.integers(0, len(balls))]
        yield int(100 * s / i)


def simulation(days: int, iterations: int, balls: list[int]):
    res = np.zeros((1201, days), dtype=int)
    
    for _ in range(iterations):
        average = average_sequence(balls)
        for d in range(days):
            a = next(average)
            res[a][d] += 1
            
    return res


def numbers_to_averages(ns: list[int]):
    res = np.empty(len(ns))
    s = 0
    for i, n in enumerate(ns):
        s += n
        res[i] = s / (i + 1)
    return res


def fraction_that_far(sim, val, day):
    delta = abs(int(val * 100) - 550)
    day_values = sim[:,day]    
    total = sum(day_values)
    good = sum(day_values[550 + delta:])  + sum(day_values[0:550 - delta])
    return good / total


def probabilities(sim, averages, skip=0):
    res = np.empty(len(averages))
    
    for i, a in enumerate(averages):
        res[i] = 100 * fraction_that_far(sim, a, i + 1 + skip)
    
    return res


NUMBERS = [8, 8, 3, 9, 5, 1, 2, 9, 10, 8, 8, 1, 9, 9, 8, 5, 1, 10, 5, 9, 6, 4, 10, 3, 10, 9, 8, 4, 8, 8, 9, 9, 1, 5, 9,
           8, 4, 1, 8, 6, 7, 8, 5, 6, 10, 9, 1, 1, 5, 6, 2, 8, 6, 5, 2, 5, 4, 10, 10, 2, 2, 4, 9, 6, 9, 9, 6, 10, 9, 10,
           8, 1, 10, 5, 8, 6, 8, 10, 4, 10, 9, 10, 7, 4, 7, 8, 6, 7, 3, 10, 3, 8, 3, 3, 6, 7, 2, 6, 9, 5, 9, 10, 6, 7,
           10, 8, 5, 8, 1, 6, 8, 3, 7, 8, 5, 1, 9, 3, 2, 6, 7, 8, 3, 9, 1, 10, 1, 8, 4, 4, 9, 2, 6, 7, 10, 7, 8, 3, 9,
           7, 3, 6, 10, 5, 1, 6, 10, 7, 8, 10, 5, 6, 8, 6, 9, 10, 2, 4, 9, 4, 3, 4, 10, 6, 6, 1, 7, 10, 3, 1, 6, 3, 6,
           9, 7, 9, 7, 9, 6, 5, 10, 8, 7, 5, 9, 8, 5, 5, 4, 7, 7, 4, 6, 1, 10, 8, 9, 7, 8, 6, 6, 6, 3, 6, 7, 7, 2, 10,
           6, 1, 5, 2, 7, 10, 2, 2, 7, 3, 1, 5, 9, 10, 2, 5, 3, 9, 9, 4, 3, 8, 8, 6, 3, 2, 5, 1, 2, 7, 7, 4, 2, 6, 7, 6,
           6, 8, 1, 5, 2, 6, 5, 5, 6, 5, 4, 10, 3, 1, 1, 3, 3, 3, 4, 6, 5, 10, 4, 3, 4, 6, 5, 7, 1, 3, 4, 6, 6, 9, 6, 4,
           7, 8, 1, 7, 6, 3, 4, 5, 7, 7, 10, 10, 7, 6, 8, 8, 5, 10, 2, 8, 7, 5, 6, 2, 10, 10, 2, 8, 4, 4, 3, 10, 3, 6,
           3, 5, 8, 5, 10, 5, 4, 4, 10, 6, 5, 4, 6, 6, 3, 2, 8, 4, 8, 10, 10, 8, 2, 8, 8, 3, 10, 10, 3, 4, 5, 8, 10, 2,
           1, 4, 7, 6, 10, 4, 7, 6, 2, 1, 10, 4, 3, 9, 8, 5, 2, 9, 10, 1, 3, 8, 10, 8, 6, 2, 9, 7, 7, 8, 10, 10, ]

In [ ]:
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from multiprocessing import Pool, cpu_count
from matplotlib.collections import LineCollection
from pathlib import Path

DAYS = 400
ITERS = 1_000_000
BALLS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
cpus = cpu_count()

with Pool(processes=cpus) as pool:
    work = [(DAYS, ITERS // cpus, BALLS)] * cpus
    data = sum(pool.starmap(simulation, work))

SKIP=100
AVERAGES = numbers_to_averages(NUMBERS)[SKIP:]
X = np.linspace(1 + SKIP, len(AVERAGES) + 1 + SKIP, len(AVERAGES))
Z = probabilities(data, AVERAGES, SKIP)

fig, ax = plt.subplots(figsize=(40, 20))

max_val = data.max()
cmap="magma"
text_color = "white"

im = ax.pcolormesh(
    np.arange(0, DAYS, 1),
    np.arange(0, 12.01, 0.01), 
    data, cmap=cmap, shading="auto", vmin=0, vmax=max_val / 2)

points = np.array([X, AVERAGES]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = plt.Normalize(Z.min(), Z.max())
lc = LineCollection(segments, cmap='PiYG', norm=norm)
lc.set_array(Z)
lc.set_linewidth(2)
line = ax.add_collection(lc)
cb = fig.colorbar(line, ax=ax)
cb.ax.yaxis.set_tick_params(color=text_color)
plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color=text_color)

ax.set_xlim([0, DAYS])
ax.set_ylim([4.50, 6.50])

plt.ylabel("среднее", fontsize=24, color=text_color)
plt.xlabel("дни", fontsize=24, color=text_color)

ax.xaxis.set_major_locator(MultipleLocator(100))
ax.xaxis.set_minor_locator(AutoMinorLocator(4))
ax.yaxis.set_major_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))

plt.yticks(fontsize=20, color=text_color)
plt.xticks(fontsize=20, color=text_color)

ax.grid(which="major", axis="both", linestyle="-", linewidth=1)
ax.grid(which="minor", axis="both", linestyle="-", linewidth=0.25)

plt.show()

image_path="./images"
Path(image_path).mkdir(exist_ok=True)
fig.savefig(f"{image_path}/average_{DAYS}d_{ITERS}i_{cmap}_{text_color}.png", facecolor="black")